# Object Detection
模型: `facebook/detr-resnet-50`  
框架: `MindSpore` + `Mindnlp`  
环境: 香橙派 AIpro（8T/16G） + Ubuntu + MindSpore 2.6.0 + MindNLP 0.4.1  
目标: 实现目标检测任务，预测图像中每个实体的类别

In [10]:
!pip install mindnlp==0.4.1 pillow sympy 

Traceback (most recent call last):
  File "/usr/local/miniconda3/envs/img/bin/pip", line 8, in <module>
    sys.exit(main())
  File "/usr/local/miniconda3/envs/img/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 78, in main
    command = create_command(cmd_name, isolated=("--isolated" in cmd_args))
  File "/usr/local/miniconda3/envs/img/lib/python3.9/site-packages/pip/_internal/commands/__init__.py", line 121, in create_command
    module = importlib.import_module(module_path)
  File "/usr/local/miniconda3/envs/img/lib/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850

In [ ]:
from PIL import Image,ImageDraw
import mindspore as ms
from mindspore import ops, Tensor
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from mindnlp.transformers import DetrImageProcessor, DetrForObjectDetection

### 加载模型

In [ ]:
# 初始化处理器和模型
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")


### 加载图像并进行推理

In [ ]:
image_path = "image.png"
image = Image.open(image_path).convert("RGB")

inputs_np = processor(images=image, return_tensors="np")
inputs = {k: Tensor(v, dtype=ms.float32) for k, v in inputs_np.items()}

outputs = model(**inputs)

### 对推理结果进行处理提取出识别结果内容

In [ ]:

target_sizes = Tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

### 展示识别结果

In [ ]:
# 创建图像副本用于绘制
image_with_boxes = image.copy()
draw = ImageDraw.Draw(image_with_boxes)

# 设置颜色列表
colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

# 绘制边界框和标签
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    # 转换坐标
    box = [round(i, 2) for i in box.tolist()]
    x_min, y_min, x_max, y_max = box
    
    # 获取类别名称和置信度
    class_name = model.config.id2label[label.item()]
    confidence = round(score.item(), 3)
    
    # 选择颜色
    color_idx = label.item() % len(colors)
    color = colors[color_idx]
    
    # 绘制边界框
    draw.rectangle([x_min, y_min, x_max, y_max], outline=color, width=3)
    
    # 绘制标签背景
    label_text = f"{class_name}: {confidence}"
    bbox = draw.textbbox((x_min, y_min), label_text)
    draw.rectangle(bbox, fill=color)
    
    # 绘制标签文本
    draw.text((x_min, y_min), label_text, fill="white")

# 使用matplotlib显示
plt.figure(figsize=(12, 8))
plt.imshow(np.array(image_with_boxes))
plt.axis('off')
plt.title('Object Detection Results', fontsize=16)
plt.tight_layout()
plt.show()

